In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, BatchNormalization, LeakyReLU, Flatten, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix


In [2]:
# Define constants
time_steps = 100   # Number of timesteps in input data
features = 10      # Number of features in each timestep
latent_dim = 64    # Dimension of latent space

# Generator (Autoencoder with Encoder1 and Decoder)
def build_generator():
    inputs = Input(shape=(time_steps, features))
    
    # Encoder 1 (LSTM layers + Fully Connected Layers)
    x = LSTM(128, return_sequences=True)(inputs)
    x = LSTM(64)(x)
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    # Latent representation (z)
    latent = Dense(latent_dim, activation='relu')(x)

    # Decoder (Symmetrical to Encoder 1)
    x = Dense(64)(latent)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dense(time_steps * features)(x)
    outputs = Reshape((time_steps, features))(x)
    
    model = Model(inputs, outputs, name="Generator")
    return model

# Discriminator (used to classify real vs fake)
def build_discriminator():
    inputs = Input(shape=(time_steps, features))
    
    x = LSTM(128, return_sequences=True)(inputs)
    x = LSTM(64)(x)
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    # Flatten and classify real vs fake
    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, x, name="Discriminator")
    return model

# Define loss functions
def generator_loss(fake_output, reconstructed_input, real_latent, reconstructed_latent, lambdas):
    bce_loss = BinaryCrossentropy(from_logits=True)
    
    # Reconstruction loss (input space, L1 loss)
    lx = tf.reduce_mean(tf.abs(reconstructed_input - real_latent))
    
    # Latent space reconstruction loss (L2 loss)
    lz = tf.reduce_mean(tf.square(reconstructed_latent - real_latent))
    
    # Adversarial loss (Binary Crossentropy)
    adversarial_loss = bce_loss(tf.ones_like(fake_output), fake_output)
    
    # Combine losses
    loss = lambdas[0] * lx + lambdas[1] * lz + lambdas[2] * adversarial_loss
    return loss

def discriminator_loss(real_output, fake_output):
    bce_loss = BinaryCrossentropy(from_logits=True)
    
    # Binary crossentropy loss for real and fake samples
    real_loss = bce_loss(tf.ones_like(real_output), real_output)
    fake_loss = bce_loss(tf.zeros_like(fake_output), fake_output)
    
    # Total discriminator loss
    return real_loss + fake_loss


In [4]:
# Build and compile the models
generator = build_generator()
discriminator = build_discriminator()

generator.summary()
discriminator.summary()
# Optimizers
gen_optimizer = Adam(0.0002, beta_1=0.5)
disc_optimizer = Adam(0.0002, beta_1=0.5)

# Placeholders for real and fake inputs
real_data = Input(shape=(time_steps, features))
fake_data = generator(real_data)

# Discriminator training
real_output = discriminator(real_data)
fake_output = discriminator(fake_data)

# Custom training loop
@tf.function
def train_step(real_data):
    # Latent vectors for reconstruction
    real_latent = generator(real_data)
    
    with tf.GradientTape(persistent=True) as tape:
        # Generate fake data
        fake_data = generator(real_data)
        
        # Discriminator outputs
        real_output = discriminator(real_data)
        fake_output = discriminator(fake_data)
        
        # Latent space reconstruction
        reconstructed_latent = generator(fake_data)
        
        # Calculate losses
        gen_loss = generator_loss(fake_output, fake_data, real_latent, reconstructed_latent, lambdas=[0.1, 0.1, 0.8])
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # Update gradients
    gradients_of_generator = tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = tape.gradient(disc_loss, discriminator.trainable_variables)
    
    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    disc_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

Model: "Generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 10)]         0         
                                                                 
 lstm_4 (LSTM)               (None, 100, 128)          71168     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 batch_normalization_3 (Batc  (None, 64)               256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 64)                0         
                                                         

In [5]:
# Training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        for real_data in dataset:
            gen_loss, disc_loss = train_step(real_data)
        
        print(f'Epoch {epoch + 1}, Gen Loss: {gen_loss}, Disc Loss: {disc_loss}')

# Anomaly score and labeling
def compute_anomaly_score(real_data):
    fake_data = generator.predict(real_data)
    
    # Latent space reconstruction loss
    latent_real = generator(real_data)
    latent_fake = generator(fake_data)
    
    anomaly_score = np.mean(np.square(latent_real - latent_fake), axis=1)
    
    return anomaly_score

def label_data(anomaly_score, thresholds):
    labels = np.zeros_like(anomaly_score)
    labels[anomaly_score < thresholds[0]] = 0  # "Good"
    labels[(anomaly_score >= thresholds[0]) & (anomaly_score < thresholds[1])] = 1  # "Watching"
    labels[(anomaly_score >= thresholds[1]) & (anomaly_score < thresholds[2])] = 2  # "Warning"
    labels[anomaly_score >= thresholds[2]] = 3  # "Fault"
    return labels

# Model Evaluation
def evaluate_model(real_data, true_labels, thresholds):
    # Compute anomaly scores
    anomaly_scores = compute_anomaly_score(real_data)
    
    # Label data based on thresholds
    predicted_labels = label_data(anomaly_scores, thresholds)
    
    # Calculate evaluation metrics
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    cm = confusion_matrix(true_labels, predicted_labels)
    
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)

# Sample dataset and training example
train_data = np.random.randn(1000, time_steps, features)
train_dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(32)

# Train the model
train(train_dataset, epochs=50)

# Evaluate the model (replace `true_labels` with actual labels)
true_labels = np.random.randint(0, 4, size=1000)
# evaluate_model(train_data, true_labels, thresholds=[0.2, 0.5, 0.7])

c:\Users\farra\anaconda3\lib\site-packages\keras\backend.py:5673: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Gen Loss: 0.650508463382721, Disc Loss: 1.1934301853179932
Epoch 2, Gen Loss: 2.7452542781829834, Disc Loss: 0.5759519338607788
Epoch 3, Gen Loss: 0.0530317984521389, Disc Loss: 3.0407092571258545
Epoch 4, Gen Loss: 0.6268647909164429, Disc Loss: 0.9519755244255066
Epoch 5, Gen Loss: 0.7185719609260559, Disc Loss: 0.8008942604064941
Epoch 6, Gen Loss: 1.5878078937530518, Disc Loss: 0.3267285227775574
Epoch 7, Gen Loss: 1.6642814874649048, Disc Loss: 0.2551543712615967
Epoch 8, Gen Loss: 2.2694976329803467, Disc Loss: 0.15700165927410126
Epoch 9, Gen Loss: 2.5059597492218018, Disc Loss: 0.12790241837501526
Epoch 10, Gen Loss: 2.66983699798584, Disc Loss: 0.11056220531463623
Epoch 11, Gen Loss: 2.7254414558410645, Disc Loss: 0.1016135960817337
Epoch 12, Gen Loss: 2.8578872680664062, Disc Loss: 0.09121386706829071
Epoch 13, Gen Loss: 3.1955230236053467, Disc Loss: 0.08169358223676682
Epoch 14, Gen Loss: 3.1780996322631836, Disc Loss: 0.02545725554227829
Epoch 15, Gen Loss: 3.2758

ValueError: Classification metrics can't handle a mix of multiclass and multiclass-multioutput targets

In [9]:
def convert_time_window(data, timesteps):
    samples = data.shape[0] - timesteps + 1  # Number of samples in the new 3D array
    variables = data.shape[1]  # Number of variables (features)

    data_3d = np.zeros((samples, timesteps, variables))

    for i in range(samples):
        data_3d[i] = data[i:i+timesteps]
    return data_3d

In [10]:
def inverse_convert_time_window(data_3d):
    samples, timesteps, variables = data_3d.shape
    data_2d = np.zeros((samples + timesteps - 1, variables))

    count = np.zeros((samples + timesteps - 1, variables))

    for i in range(samples):
        data_2d[i:i+timesteps] += data_3d[i]
        count[i:i+timesteps] += 1

    # Average the overlapping segments
    data_2d /= count
    return data_2d

In [17]:
# train_data_2d = inverse_convert_time_window(train_data)

fake_data = generator.predict(train_data)

fake_data_2d = np.mean(fake_data, axis=1)

32/32 [==============================] - 1s 24ms/step


In [21]:
fake_data = generator.predict(train_data)

# Latent space reconstruction loss
latent_real = generator(train_data)
latent_fake = generator(fake_data)

latent_real_2d = inverse_convert_time_window(latent_real)
latent_fake_2d = inverse_convert_time_window(latent_fake)

anomaly_score = np.mean(np.square(latent_real_2d - latent_fake_2d), axis=1)

32/32 [==============================] - 1s 26ms/step


In [23]:
predicted_labels = label_data(anomaly_score, thresholds=[0.2, 0.5, 0.7])

# Calculate evaluation metrics
precision = precision_score(true_labels, predicted_labels[:1000], average='weighted')
recall = recall_score(true_labels, predicted_labels[:1000], average='weighted')
f1 = f1_score(true_labels, predicted_labels[:1000], average='weighted')
cm = confusion_matrix(true_labels, predicted_labels[:1000])

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
print("Confusion Matrix:")
print(cm)

Precision: 0.0531, Recall: 0.2300, F1-Score: 0.0862
Confusion Matrix:
[[230   0   0   0]
 [240   0   0   0]
 [262   3   0   0]
 [265   0   0   0]]


c:\Users\farra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
# fake_data = generator.predict(train_data)

# anomaly_scores = compute_anomaly_score(train_data)
# predicted_labels = label_data(anomaly_scores, thresholds=[0.2, 0.5, 0.7])

32/32 [==============================] - 1s 26ms/step


In [8]:
# # Set device
# device = "/gpu:0" if tf.config.list_physical_devices('GPU') else "/cpu:0"

# # LSTM Encoder
# class LSTMEncoder(tf.keras.Model):
#     def __init__(self, input_size, hidden_size, latent_size, num_layers):
#         super(LSTMEncoder, self).__init__()
#         self.lstm = layers.LSTM(hidden_size, num_layers, return_sequences=False, return_state=True)
#         self.fc = models.Sequential([
#             layers.Dense(latent_size),
#             layers.BatchNormalization(),
#             layers.ReLU()
#         ])
        
#     def call(self, x):
#         _, h, _ = self.lstm(x)
#         z = self.fc(h)
#         return z

# # Decoder (Generator)
# class LSTMDecoder(tf.keras.Model):
#     def __init__(self, latent_size, hidden_size, output_size, num_layers):
#         super(LSTMDecoder, self).__init__()
#         self.fc = layers.Dense(hidden_size)
#         self.lstm = layers.LSTM(output_size, num_layers, activation='relu', return_sequences=True)
        
#     def call(self, z, seq_len):
#         h = tf.nn.relu(self.fc(z))
#         h = tf.expand_dims(h, 1)
#         h = tf.tile(h, [1, seq_len, 1])  # Expand to sequence length
#         out = self.lstm(h)
#         return out

# # Discriminator
# class Discriminator(tf.keras.Model):
#     def __init__(self, input_size, hidden_size, num_layers):
#         super(Discriminator, self).__init__()
#         self.lstm = layers.LSTM(hidden_size, num_layers, return_state=False, activation='relu', return_sequences=False)
#         self.fc = layers.Dense(1, activation='sigmoid')
        
#     def call(self, x):
#         h = self.lstm(x)
#         out = self.fc(h)
#         return out

# # LSTM-GAN Model
# class LSTM_GAN(tf.keras.Model):
#     def __init__(self, input_size, hidden_size, latent_size, num_layers):
#         super(LSTM_GAN, self).__init__()
#         self.encoder = LSTMEncoder(input_size, hidden_size, latent_size, num_layers)
#         self.decoder = LSTMDecoder(latent_size, hidden_size, input_size, num_layers)
#         self.discriminator = Discriminator(input_size, hidden_size, num_layers)
    
#     def call(self, x):
#         z = self.encoder(x)
#         reconstructed_x = self.decoder(z, tf.shape(x)[1])
#         return reconstructed_x


In [3]:
# # Loss Functions
# def generator_loss(reconstructed_x, real_x, z, z_hat, discriminator_out, w1=1.0, w2=1.0, w3=1.0):
#     l1_loss = tf.reduce_mean(tf.abs(reconstructed_x - real_x))
#     l2_loss = tf.reduce_mean(tf.square(z - z_hat))
#     adv_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(discriminator_out), discriminator_out)
#     return w1 * l1_loss + w2 * l2_loss + w3 * adv_loss

# def discriminator_loss(real_output, fake_output):
#     real_loss = tf.keras.losses.BinaryCrossentropy()(tf.ones_like(real_output), real_output)
#     fake_loss = tf.keras.losses.BinaryCrossentropy()(tf.zeros_like(fake_output), fake_output)
#     return real_loss + fake_loss

# # Data Preparation (synthetic for demo purposes)
# def generate_synthetic_data(num_samples, seq_len, input_size):
#     data = np.sin(np.linspace(0, 100, num_samples * seq_len).reshape(num_samples, seq_len, input_size))
#     return data + 0.05 * np.random.randn(num_samples, seq_len, input_size)

# # Labeling Reconstructed Data
# def label_reconstructed_data(real_x, reconstructed_x, thresholds):
#     errors = tf.reduce_mean(tf.abs(real_x - reconstructed_x), axis=[1, 2]).numpy()
#     labels = np.zeros(len(errors))
#     labels[errors >= thresholds[2]] = 3  # Fault
#     labels[(errors >= thresholds[1]) & (errors < thresholds[2])] = 2  # Warning
#     labels[(errors >= thresholds[0]) & (errors < thresholds[1])] = 1  # Watching
#     labels[errors < thresholds[0]] = 0  # Good
#     return labels

# # Model Evaluation
# def evaluate_model(real_labels, predicted_labels):
#     print("Confusion Matrix:\n", confusion_matrix(real_labels, predicted_labels))
#     print("Classification Report:\n", classification_report(real_labels, predicted_labels))

# # Main training loop
# def train_lstm_gan(model, dataset, epochs, input_size, hidden_size, latent_size, num_layers):
#     optimizer_G = tf.keras.optimizers.Adam(learning_rate=0.001)
#     optimizer_D = tf.keras.optimizers.Adam(learning_rate=0.001)
    
#     for epoch in range(epochs):
#         for data, labels in dataset:
#             real_x = data
            
#             with tf.GradientTape() as tape_D, tf.GradientTape() as tape_G:
#                 # Train Discriminator
#                 z = model.encoder(real_x)
#                 reconstructed_x = model.decoder(z, tf.shape(real_x)[1])
#                 fake_output = model.discriminator(reconstructed_x)
#                 real_output = model.discriminator(real_x)
#                 d_loss = discriminator_loss(real_output, fake_output)

#                 # Train Generator
#                 z_hat = model.encoder(reconstructed_x)
#                 g_loss = generator_loss(reconstructed_x, real_x, z, z_hat, fake_output)
            
#             gradients_D = tape_D.gradient(d_loss, model.discriminator.trainable_variables)
#             optimizer_D.apply_gradients(zip(gradients_D, model.discriminator.trainable_variables))
            
#             gradients_G = tape_G.gradient(g_loss, model.encoder.trainable_variables + model.decoder.trainable_variables)
#             optimizer_G.apply_gradients(zip(gradients_G, model.encoder.trainable_variables + model.decoder.trainable_variables))
        
#         if epoch % 10 == 0:
#             print(f'Epoch [{epoch}/{epochs}], Discriminator Loss: {d_loss:.4f}, Generator Loss: {g_loss:.4f}')
#     return model


In [ ]:
# # Parameters and Data
# input_size = 1
# hidden_size = 64
# latent_size = 32
# num_layers = 2
# num_samples = 1000
# seq_len = 50
# epochs = 100

# data = generate_synthetic_data(num_samples, seq_len, input_size)
# labels = np.zeros(num_samples)  # Synthetic labels

# # Prepare Dataset and Dataloader
# dataset = tf.data.Dataset.from_tensor_slices((data, labels)).batch(32).shuffle(100)

# # Initialize and Train the Model
# with tf.device(device):
#     model = LSTM_GAN(input_size, hidden_size, latent_size, num_layers)
#     trained_model = train_lstm_gan(model, dataset, epochs, input_size, hidden_size, latent_size, num_layers)

# # Evaluate the Model
# with tf.device(device):
#     real_data = tf.convert_to_tensor(data, dtype=tf.float32)
#     reconstructed_data = trained_model(real_data)
#     thresholds = [0.02, 0.05, 0.1]  # Example thresholds for labeling
#     predicted_labels = label_reconstructed_data(real_data, reconstructed_data, thresholds)
#     evaluate_model(labels, predicted_labels)
